# Interactive Julia for *A Simple Nearly-optimal Restart Scheme for Speeding-Up First Order Methods* #

### James Renegar, Benjamin Grimmer ###
___

This notebook follows along the ideas presented in [[1]](https://arxiv.org/abs/1803.00151). Its purpose is to give a simple and understandable implementation of the algorithms and concepts therein. Throughout this document, we may reference equations, lemmas, and theorems using the numbering given in that paper. 

The general outline of this document is the following:

1. Implementation of Basic Synchronous Scheme
2. Solving Some Simple Problems:
    * --- via Subgradient Method
    * --- via Accelerated Method
3. Implementation of Parallel Asynchronous Scheme
4. Solving an Interesting Problems:
    * --- via Inexact Proximal Point Method

**License**: Creative Commons License

This work is licensed under a Creative Commons Attribution-ShareAlike 4.0 International License (http://creativecommons.org/licenses/by-sa/4.0/).

An MIT License for the underlying software is provided in the separate LICENSE file.

**Disclaimer**: We make no claim that the code below is optimized or the most efficient version possible. Instead our goal is to give the simplest implementation possible. Speedups are certainly possible using optimized linear algebra packages and by taking advantage of sparsity and problem specific structure.

Any questions or feedback on this document can be sent to Ben Grimmer at *bdg79@cornell.edu*. 

[1] "A Simple Nearly-Optimal Restart Scheme For Speeding-Up First Order Methods", https://arxiv.org/abs/1803.00151. James Renegar, Benjamin Grimmer. First posted March 1st, 2018.

# Implementation of Basic Synchronous Scheme #

# Solving Some Simple Problems #

# Implementation of Parallel Asynchronous Scheme #

# Solving an Interesting Problem #